# Routing with FastAPI

When you work in a project, your code will be partitioned into smaller scripts, so your program is more organized. In FastAPI we can do the same using Routers. As an example, let's say that you are adding two GET responses to your API, one for a regular DOB request, and another for giving an icon to the tab we will work in:

In [ ]:
%%python
import fastapi
import uvicorn

api = fastapi.FastAPI()

@api.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'

@api.get('/api/dob')
def dob():
    return 'Some random DOB'
    
if __name__ == '__main__':
    uvicorn.run(api, port=8008, host='127.0.0.1')

You can go to `http://127.0.0.1:8008/api/dob` and see the output:
<p align=center><img src=images/Routing_1.png width=300></p>

One way to separate these functions would be having one script for the home page, and another one for additional pages (`/api/dob`)

```
root/
│
├── home.py
├── api
│   └── dob_api.py
└── main.py
```

In [ ]:
# views/home.py

import fastapi

router = fastapi.APIRouter()


@router.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'


In [ ]:
# api/dob_api.py

import fastapi

router = fastapi.APIRouter()

@router.get('/api/dob')
def dob():
    return 'Some random DOB'

In [ ]:
# main.py
import fastapi
import uvicorn
import home

from api import dob_api

api = fastapi.FastAPI()


def configure_routing():
    api.include_router(home.router)
    api.include_router(dob_api.router)


if __name__ == '__main__':
    configure_routing()
    uvicorn.run(api, port=8000, host='127.0.0.1')


Observe that in `home.py` and `api/dob_api.py`, we included `router = fastapi.APIRouter()`, so all files have a common node to look for all requests.


## Try it out

1. Create a directory named views
2. Add a file named home.py in said directory
3. Add the code above
4. Create a directory named api
5. Add a file named dob_api.py in said directory
6. Add the code above to dob_api.py
7. Create a file named main.py ann add the code above
8. Run the code and check everything works
9. Tweak dob_api, so `/api/dob` can accept two query parameters: first name and last name

# Add Pydantic model

Your API will accept request parameters, and Pydantic will be responsible for casting those parameters to the correct format. In this example, we can accept the first name and the last name. 

Pydantic allows you to create a class where each attribute will be casted to the specified data type

It is recommended that you refresh your knowledge of Python type hinting and Pydantic before continuing with this section.

FastAPI will use this type hinting to perform the transformation of the query parameters into the specified data type we put as a hint.

In [ ]:
from pydantic import BaseModel
from typing import Optional

class Celebrity(BaseModel):
    '''
    This class gives some information about a celebrity. It is intended to be 
    used with the FastAPI example

    Attributes
    ----------
    first_name: str
        The first name of the celebrity
    last_name: str
        The last name of the celebrity
    city: bool
        If True, the API will also returns the city where the celebrity was born
    '''
    first_name: str
    last_name: str
    city: bool = False

uma_thurman = Celebrity(first_name='Uma', last_name='Thurman')    

Let's create another directory to store our models:

```
root/
│
├── home.py
├── api
│   └── dob_api.py
├── models
│   └── celebrities.py
└── main.py
```

Then, we can use this Pydantic model in our FastAPI like shown below:

In [ ]:
# models/celebrities.py

from pydantic import BaseModel
from typing import Optional

class Celebrity(BaseModel):
    '''
    This class gives some information about a celebrity. It is intended to be 
    used with the FastAPI example

    Attributes
    ----------
    first_name: str
        The first name of the celebrity
    last_name: str
        The last name of the celebrity
    city: bool
        If True, the API will also returns the city where the celebrity was born
    '''
    first_name: str
    last_name: str
    city: bool = False

In [ ]:
# api/dob_api.py

import fastapi
from fastapi import Depends

from models.celebrities import Celebrity

router = fastapi.APIRouter()


@router.get('/api/dob/{first_name}')
def dob(celebrity: Celebrity = Depends()):
    full_name = f'{celebrity.first_name} {celebrity.last_name}'
    if celebrity.city:
        report = f'{full_name} was born a random day in a random city'
    else:
        report = f'{full_name} was born a random day'
    return report



One change that you can see is the `Depends` class in the celebrity when used as an argument in `dob`. This will tell the dob function that the variables inside it depend on an external class, so it will know that the attributes of the class are going to be query parameters.

With this, the following URL [http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman](http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman) will return this output:

<p align=center><img src=images/Routing_2.png width=400></p>

Right now, the DOB API is not doing what it's expected to do. Let's create the real function that will scrape the data about the celebrity so that we can return the date through our API. The function that will contain this service will be in the `services` directory:

```
root/
│
├── home.py
├── api
│   └── dob_api.py
├── models
│   └── celebrities.py
├── services
│   └── dob_service.py
└── main.py
```

In [ ]:
# services/dob_service.py
import requests
from bs4 import BeautifulSoup
import json
import re

def get_dob(first_name: str, last_name: str, city: bool=False):
    infobox_data = get_infobox(first_name, last_name)
    if not infobox_data:
        return None
    birthday = infobox_data.find('span', {'class': 'bday'})

    report = {'first name': first_name,
              'last_name': last_name,
              'Date of Birth': birthday.text}
    if city:
        birthplace = infobox_data.find('div', {'class': 'birthplace'})
        report['City'] = birthplace.text

    return report

def get_infobox(first_name: str, last_name: str):
    r = requests.get(f'https://en.wikipedia.org/wiki/{first_name}_{last_name}')
    soup = BeautifulSoup(r.text, 'html.parser')
    if soup.find_all('b', text=re.compile('Wikipedia does not have an article with this exact name')):
        return None
    celeb_infobox = soup.find('table', {'class': 'infobox biography vcard'})
    return celeb_infobox.find('td', {'class': 'infobox-data'})

In [ ]:
# api/dob_api.py

import fastapi
from fastapi import Depends
import json
from models.celebrities import Celebrity
# from services.dob_service import get_dob
from services.dob_service_async import get_dob
router = fastapi.APIRouter()

@router.get('/api/dob/{first_name}')
def dob(celebrity: Celebrity = Depends()):
    report = get_dob(celebrity.first_name, celebrity.last_name, celebrity.city)
    return fastapi.Response(content=json.dumps(report),
                            media_type='application/json')

If you go to the following URL [http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True](http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True), you will get a real API response!
<p align=center><img src=images/Routing_3.png width=500></p>

Also, you can use the requests library to get the json object that has been uploaded to check that everything worked properly:

In [ ]:
requests.get('http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True').json()

# Summary

- You learned how to route API requests to a single node called in main and how to use Pydantic Model in FastAPI 
    